In [1]:
import tensorflow as tf
import numpy as np
print("TensorFlow version: ",format(tf.__version__))
import Layers
from pathlib import Path
import DataSets as ds

TensorFlow version:  2.0.0


### chargement des données

In [2]:
LoadModel = False
experiment_size = 10
train = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_%dk.bin"%experiment_size), \
                   Path("/home/romain/jupyter_mla/Databases/gender_%dk.bin"%experiment_size), 1000*experiment_size)
test = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_test10k.bin"), \
     Path("/home/romain/jupyter_mla/Databases/gender_test10k.bin"), 10000)

nb data =  10000
nb data =  10000


### classes pour définir les couches et la structure du réseau

In [3]:
class ConvNeuralNet(tf.Module):
    def __init__(self):
        self.unflat = Layers.unflat('unflat',48, 48, 1)
        self.cv1 = Layers.conv('conv_1', output_dim=3, filterSize=3, stride=1)
        self.mp = Layers.maxpool('pool', 2)
        self.cv2 = Layers.conv('conv_2', output_dim=6, filterSize=3, stride=1)
        self.cv3 = Layers.conv('conv_3', output_dim=12, filterSize=3, stride=1)
        self.flat = Layers.flat()
        self.fc = Layers.fc('fc', 2)

    def __call__(self, x, log_summary):
        x = self.unflat(x, log_summary)
        x = self.cv1(x, log_summary)
        x = self.mp(x)
        x = self.cv2(x, log_summary)
        x = self.mp(x)
        x = self.cv3(x, log_summary)
        x = self.mp(x)
        x = self.flat(x)
        x = self.fc(x, log_summary)
        return x

In [4]:
def train_one_iter(model, optimizer, image, label, log_summary):
    with tf.GradientTape() as tape:
        y = model(image,log_summary)
        y = tf.nn.log_softmax(y)
        diff = label * y
        loss = -tf.reduce_sum(diff)
        if log_summary:
            tf.summary.scalar('cross entropy', loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

### Résumé du modèle

In [5]:
print ("-----------------------------------------------------")
print ("----------------------- %dk -------------------------"%experiment_size)
print ("-----------------------------------------------------")

-----------------------------------------------------
----------------------- 10k -------------------------
-----------------------------------------------------


In [6]:
train_summary_writer = tf.summary.create_file_writer('logs %dk'%experiment_size)
optimizer = tf.optimizers.Adam(1e-3)
simple_cnn = ConvNeuralNet()

def unflat unflat ? => 48 48 1


In [7]:
if LoadModel:
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=simple_cnn)
    ckpt.restore('/home/romain/jupyter_mla/saved_model-1')

In [8]:
for iter in range(1000):
    tf.summary.experimental.set_step(iter)
    
    if iter % 500 == 0:
        with train_summary_writer.as_default():
            acc1 = train.mean_accuracy(simple_cnn) * 100
            acc2 = test.mean_accuracy(simple_cnn) * 100
            print("iter= %6d accuracy - train= %.2f%% - test= %.2f%%" % (iter, acc1, acc2))

    ima, lab = train.NextTrainingBatch()
    with train_summary_writer.as_default():
        loss = train_one_iter(simple_cnn, optimizer, ima, lab, iter % 10 == 0)

    if iter % 100 == 0:
        print("iter= %6d - loss= %f" % (iter, loss))

build conv conv_1 3x3  1 => 3
build conv conv_2 3x3  3 => 6
build conv conv_3 3x3  6 => 12
build fc fc  432 => 2
iter=      0 accuracy - train= 47.60% - test= 53.97%
iter=      0 - loss= 88.643227


KeyboardInterrupt: 

In [ ]:
if not LoadModel:
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=simple_cnn)
    ckpt.save('/home/romain/jupyter_mla/saved_model')